# Situações dos Projetos no pipeline do Salic

**ISSUE:** [#32](https://github.com/lappis-unb/salic-ml/issues/32)

Este notebook é uma continuação direta deste [notebook](https://github.com/lappis-unb/salic-ml/blob/master/notebooks/exploratory/project_stage_on_pipeline-v0.1.ipynb).

### Introdução

**Etapas de Projeto:** No Salic um projeto pode passar pelas seguintes etapas, em ordem:

- Admissibilidade
- Capitação e Execução
- Prestação de Contas
- Análise do Objeto
- Análise Financeira

Se por algum motivo o projeto for finalizado prematuramente ele não irá passar por todas as etapas.

A "situação" de um projeto em um dado momento é um código ('A42', por exemplo) que está associada a uma condição do projeto. É conhecido que algumas situações podem ocorrer unicamente em etapas específicas do Salic, por exemplo a situação 'A02', que significa'Projeto Cadastrado', e está associada a etapa de Admissibilidade de um projeto no Salic. Isto permite rastrear o histórico de quais etapas um projeto esteve, incluindo as datas de início e fim de cada etapa.

### Objetivos

Identificar o histórico de situações de um projeto, incluindo as datas de início e fim em cada situação.

## Prepararando os dados

**Recarregar automaticamente os módulos**

In [128]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Importando módulos python**

In [129]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from salicml.utils.dates import Dates
from salicml.models.projects import Projects

PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir, os.pardir))
DATA_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')

**Carregando os dados da tabela de Projetos**

In [130]:
cols = ['IdPRONAC', 'DtProtocolo', 'DtInicioExecucao', 'DtFimExecucao', 'Situacao', 'AnoProjeto', 'Sequencial']
projects_csv = os.path.join(DATA_FOLDER, 'projetos.csv')

dt_projects = pd.read_csv(projects_csv, usecols=cols, low_memory=False)
date_column = 'DtProtocolo'
dt_projects[date_column] = pd.to_datetime(dt_projects[date_column], format = Dates.DATE_INPUT_FORMAT)
dt_projects.head()

,IdPRONAC,AnoProjeto,Sequencial,Situacao,DtProtocolo,DtInicioExecucao,DtFimExecucao
0,1,08,7079,E63,2008-09-12,2010-01-01 00:00:00,2010-12-31 00:00:00
1,2,08,7080,G43,2008-09-12,NaN,NaN
2,3,08,7081,K00,2008-09-12,NaN,NaN
3,4,08,7082,E25,2008-09-03,NaN,NaN
4,5,08,7083,E36,2008-09-12,NaN,NaN


### Filtrando os dados

É conhecido que dados muito antigos (de 1992, por exemplo) podem ser inconsistentes, então vamos filtrar os dados a partir de uma data de início.

In [131]:
from datetime import datetime

START_DATE = datetime(day=1, month=1, year=2013)

dt_projects = dt_projects[dt_projects.loc[:, date_column] >= START_DATE]
print(min(dt_projects[date_column]))
dt_projects.head()

2013-01-02 10:59:07


,IdPRONAC,AnoProjeto,Sequencial,Situacao,DtProtocolo,DtInicioExecucao,DtFimExecucao
158164,159372,13,0001,A42,2013-01-02 10:59:07,2013-07-01 00:00:00,2013-09-30 00:00:00
158165,159373,13,0002,E24,2013-01-02 18:53:06,2014-01-16 00:00:00,2015-07-31 00:00:00
158166,159374,13,0003,E65,2013-01-02 18:59:38,2013-03-05 00:00:00,2014-01-05 00:00:00
158167,159375,13,0004,E63,2013-01-03 14:17:59,2014-01-07 10:09:08,2014-12-02 00:00:00
158168,159376,13,0005,E65,2013-01-04 10:47:03,2013-05-17 00:00:00,2015-12-31 00:00:00


### Situção de um projeto

A tabela `projetos.csv` guarda apenas última ou a situação atual de um projeto, ou seja, não é possível a partir desta tabela construir o histórico de situações de um projeto.
O objetivo agora é investigar se as tabelas`tbProjetoSituacaoAnterior` e `HistoricoSituacao` fornecem dados suficientes para construir o histŕico de situações de um projeto.

In [132]:
last_situation_path = os.path.join(DATA_FOLDER, 'tb_projeto_situacao_anterior.csv')

dt_last_situation = pd.read_csv(last_situation_path, low_memory=False)
dt_last_situation.head()

,idProjetoSituacaoAnterior,idPronac,cdSituacaoAnterior,dtSituacaoAnterior,dsProvidenciaAnterior,idOrgaoAnterior,idUsuarioAnterior,idAcao,tpAcao,dtAcao,idUsuario,stEstado
0,1,203340,E12,2018-01-01 05:00:27,Projeto com prazo de captação expirada.,272,236,132176.0,1,2018-01-02 09:59:26,236,1
1,2,216162,E10,2018-01-01 05:00:22,Situação corrigida.,272,236,146772.0,1,2018-01-02 10:07:56,236,1
2,3,207792,E12,2018-01-01 05:00:28,Projeto com prazo de captação expirada.,272,236,139891.0,1,2018-01-02 10:17:26,236,1
3,4,215231,E90,2017-12-30 05:00:18,Projeto liberado para o proponente adequar à r...,262,236,142085.0,1,2018-01-02 10:33:19,236,1
4,5,201262,E12,2018-01-01 05:00:31,Projeto com prazo de captação expirada.,272,236,136962.0,1,2018-01-02 10:39:09,236,1


In [ ]:
historic_situation_path = os.path.join(DATA_FOLDER, 'historico_situacao.csv')
DT_SITUACAO = 'DtSituacao'

historic_cols = ['AnoProjeto', 'Sequencial', 'DtSituacao', 'Situacao']
dt_historic_situation = pd.read_csv(historic_situation_path, usecols=historic_cols, low_memory=False)
dt_historic_situation.head()

In [ ]:
START_DATE = dt_historic_situation[DT_SITUACAO] >= '2013-01-01 00:00:00'
END_DATE = dt_historic_situation[DT_SITUACAO] <= '2030-01-01 00:00:00'
dt_historic_situation = dt_historic_situation[START_DATE & END_DATE]
print(len(dt_historic_situation))
dt_historic_situation.head()

**Criando a columna `idPRONAC` na tabela de historico de situações**

A tabela `historico_situacao.csv` não possui uma coluna de `idPronac`. Porém ela possui as colunas `Sequencial` e `AnoProjeto` que são suficientes para recuperar o `idPronac` de um projeto, utilizando a tabela `projetos.csv`.

In [ ]:
ANO_PROJETO, SEQUENCIAL = 'AnoProjeto', 'Sequencial'

display(dt_projects.columns)
display(dt_historic_situation.columns)

In [ ]:
merged = pd.merge(dt_historic_situation, dt_projects, how='inner', on=[ANO_PROJETO, SEQUENCIAL])

display(merged.head())

ID_PRONAC = 'IdPRONAC'

print('projects.size = {}'.format(len(dt_projects)))
print('merged.size = {}'.format(len(merged)))

dt_historic_situation = merged
dt_historic_situation.head()

**Situação_x, Situação_y**

O join das tabelas `dt_projects` e `dt_historic_situation` teve duas novas colunas `Situacao_x` e `Situacao_y`. O surgimento destas colunas se deve ao fato de que as duas tabelas tem uma coluna com o mesmo nome (`Situacao`), e o sufixo `_x` se refere a tabela da esquerda (historic_situation), e o sufixo `_y` se refere a tabela da direita (projects). É importante destacar os seguintes significados para estas colunas:

| Coluna        | Significado   |
| ------------- |:-------------:|
| Situacao_x | Situação do projeto na data desta situação (DtSituacao) |
| Situacao_y | Situacao atual do projeto, ou a última situação do projeto |

In [ ]:
dt_historic_situation.rename(columns={'Situacao_x': 'Situacao', 'Situacao_y': 'SituacaoAtual'}, inplace=True)
dt_historic_situation.head(2)

### Conclusões

A tabela `historic_situation.csv` possui a maior parte dos dados úteis para rastrear o histórico de situações de um projeto, com exceção do dado "idPronac". A partir do `AnoProjeto` e `Sequencial` é possível descobrir o idPronac de um projeto em especico. Ou seja, dado um `idPronac` agora é possível ter o histórico de situações deste projeto.

#### Contribuições futuras

A próxima contribuição imediata é identificar quais situações estão associadas unicamente a uma etapa do projeto e juntar este notebook com estes resultados, o que será suficiente para identificar o histórico de etapas de um projeto qualquer.